In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image, make_grid
import os

# Hyperparameters
batch_size = 128
z_dim = 100  # Size of the noise vector
image_size = 28
channels = 1
epochs = 50
lr = 0.0002
beta1 = 0.5  # Adam optimizer beta1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create output folder
os.makedirs("generated_images", exist_ok=True)

In [2]:
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize between [-1, 1]
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=True, download=True, transform=transform),
    batch_size=batch_size,
    shuffle=True
)

100%|██████████| 9.91M/9.91M [00:00<00:00, 58.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.83MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.8MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.84MB/s]


In [3]:
class Generator(nn.Module):
    def __init__(self, z_dim):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: (N, z_dim, 1, 1)
            nn.ConvTranspose2d(z_dim, 256, 7, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 1, 4, 2, 1, bias=False),
            nn.Tanh()  # Output range [-1, 1]
        )

    def forward(self, z):
        return self.net(z)

In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.net = nn.Sequential(
            # Input: (N, 1, 28, 28)
            nn.Conv2d(1, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        return self.net(img)

In [5]:
generator = Generator(z_dim).to(device)
discriminator = Discriminator().to(device)

# Binary Cross Entropy loss
criterion = nn.BCELoss()

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

In [6]:
def generate_and_save_images(epoch):
    generator.eval()
    with torch.no_grad():
        z = torch.randn(64, z_dim, 1, 1).to(device)
        fake_images = generator(z)
        fake_images = fake_images * 0.5 + 0.5  # Denormalize to [0,1]
        save_image(fake_images, f"generated_images/sample_epoch_{epoch}.png", nrow=8)
    generator.train()

In [7]:
k = 3  # Generator updates per iteration
p = 1  # Discriminator updates per iteration

In [8]:
for epoch in range(1, epochs + 1):
    for i, (real_imgs, _) in enumerate(train_loader):
        batch_size_curr = real_imgs.size(0)
        real_imgs = real_imgs.to(device)

        real = torch.ones(batch_size_curr, 1, device=device)
        fake = torch.zeros(batch_size_curr, 1, device=device)

        ### ---- Train Discriminator p times ---- ###
        for _ in range(p):
            z = torch.randn(batch_size_curr, z_dim, 1, 1, device=device)
            fake_imgs = generator(z)

            # Real
            real_validity = discriminator(real_imgs)
            d_real_loss = criterion(real_validity, real)

            # Fake
            fake_validity = discriminator(fake_imgs.detach())
            d_fake_loss = criterion(fake_validity, fake)

            d_loss = d_real_loss + d_fake_loss

            optimizer_D.zero_grad()
            d_loss.backward()
            optimizer_D.step()

        ### ---- Train Generator k times ---- ###
        for _ in range(k):
            z = torch.randn(batch_size_curr, z_dim, 1, 1, device=device)
            fake_imgs = generator(z)

            validity = discriminator(fake_imgs)
            g_loss = criterion(validity, real)  # fool D → label as real

            optimizer_G.zero_grad()
            g_loss.backward()
            optimizer_G.step()

        if i % 200 == 0:
            print(f"[Epoch {epoch}/{epochs}] [Batch {i}/{len(train_loader)}] "
                  f"D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")

    # Save sample images
    generator.eval()
    with torch.no_grad():
        z = torch.randn(64, z_dim, 1, 1, device=device)
        samples = generator(z)
        samples = samples * 0.5 + 0.5  # Denormalize
        save_image(samples, f"generated_images/epoch_{epoch}.png", nrow=8)
    generator.train()

[Epoch 1/50] [Batch 0/469] D Loss: 1.5404 | G Loss: 0.3703
[Epoch 1/50] [Batch 200/469] D Loss: 1.6446 | G Loss: 0.7118
[Epoch 1/50] [Batch 400/469] D Loss: 1.1452 | G Loss: 0.7580
[Epoch 2/50] [Batch 0/469] D Loss: 1.1033 | G Loss: 0.7056
[Epoch 2/50] [Batch 200/469] D Loss: 1.0716 | G Loss: 1.0414
[Epoch 2/50] [Batch 400/469] D Loss: 1.0872 | G Loss: 0.9782
[Epoch 3/50] [Batch 0/469] D Loss: 1.0503 | G Loss: 1.1495
[Epoch 3/50] [Batch 200/469] D Loss: 1.1115 | G Loss: 0.8858
[Epoch 3/50] [Batch 400/469] D Loss: 1.1566 | G Loss: 0.6472
[Epoch 4/50] [Batch 0/469] D Loss: 1.0953 | G Loss: 1.0072
[Epoch 4/50] [Batch 200/469] D Loss: 1.0822 | G Loss: 0.8813
[Epoch 4/50] [Batch 400/469] D Loss: 1.1214 | G Loss: 0.8801
[Epoch 5/50] [Batch 0/469] D Loss: 1.1151 | G Loss: 1.0809
[Epoch 5/50] [Batch 200/469] D Loss: 1.1926 | G Loss: 1.1789
[Epoch 5/50] [Batch 400/469] D Loss: 1.1351 | G Loss: 0.9392
[Epoch 6/50] [Batch 0/469] D Loss: 1.0255 | G Loss: 1.1095
[Epoch 6/50] [Batch 200/469] D Loss:

In [9]:
import os
from torchvision.utils import save_image

def generate_images_dcgan_individually(generator, num_samples=512, save_dir="generated_images", prefix="img"):
    """
    Generates images using a DCGAN generator and saves each one individually.

    Args:
        generator (nn.Module): Trained DCGAN generator model
        num_samples (int): Number of images to generate
        save_dir (str): Directory to save generated images
        prefix (str): Filename prefix for saved images
    """
    generator.eval()
    z = torch.randn(num_samples, z_dim, 1, 1).to(device)

    # Ensure output directory exists
    os.makedirs(save_dir, exist_ok=True)

    with torch.no_grad():
        gen_imgs = generator(z)
        gen_imgs = gen_imgs * 0.5 + 0.5  # Denormalize from [-1, 1] to [0, 1]

        for i, img in enumerate(gen_imgs):
            save_path = os.path.join(save_dir, f"{prefix}_{i:04d}.png")
            save_image(img, save_path)

    print(f"Saved {num_samples} images to '{save_dir}'")
    return gen_imgs

In [10]:
generate_images_dcgan_individually(generator, num_samples=1024)

Saved 1024 images to 'generated_images'


tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]]],


        [[[4.0561e-05, 1.9363e-04, 2.8775e-03,  ..., 7.1526e-07,
           1.3059e-04, 6.4731e-05],
          [4.3213e-06, 3.4660e-05, 9.6858e-06,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [5.6624e-07, 1.2368e-05, 2.5123e-05,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [1.8686e-05, 3.5763e-07, 4.6492e-06,  ..., 2.79

In [16]:
def save_random_mnist_images(save_dir="real_images/mnist",
                             num_samples=1024,
                             prefix="mnist",
                             batch_size=128):

    os.makedirs(save_dir, exist_ok=True)
    torch.manual_seed(42)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    dataset = datasets.MNIST(root="./data", train=True,
                             transform=transform, download=True)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    saved = 0
    for batch_imgs, _ in loader:
        for img in batch_imgs:
            if saved >= num_samples:
                print(f"✅ Saved {num_samples} MNIST images to '{save_dir}'")
                return

            img = img.repeat(3, 1, 1)
            save_image(img, os.path.join(save_dir, f"{prefix}_{saved:04d}.png"))
            saved += 1


In [18]:
!mkdir -p real_images/mnist
!mkdir -p generated_images/fake

In [19]:
save_random_mnist_images(save_dir="real_images/mnist")

✅ Saved 1024 MNIST images to 'real_images/mnist'


In [20]:
import os

print(os.listdir("real_images"))          # MUST show ['mnist']
print(os.listdir("real_images/mnist")[:5])  # MUST show images


['mnist_0614.png', 'mnist_0607.png', 'mnist_0610.png', 'mnist_0542.png', 'mnist_0120.png', 'mnist_0079.png', 'mnist_0609.png', 'mnist_0381.png', 'mnist_0974.png', 'mnist_0669.png', 'mnist_0437.png', 'mnist_0404.png', 'mnist_0238.png', 'mnist_0151.png', 'mnist_0130.png', 'mnist_0652.png', 'mnist_0665.png', 'mnist_0450.png', 'mnist_0195.png', 'mnist_0243.png', 'mnist_0872.png', 'mnist_0374.png', 'mnist_0915.png', 'mnist_0329.png', 'mnist_0559.png', 'mnist_0646.png', 'mnist_0225.png', 'mnist_0581.png', 'mnist_0279.png', 'mnist_0449.png', 'mnist_0034.png', 'mnist_0327.png', 'mnist_0483.png', 'mnist_0308.png', 'mnist_0842.png', 'mnist_0481.png', 'mnist_0286.png', 'mnist_0627.png', 'mnist_0923.png', 'mnist_0001.png', 'mnist_0246.png', 'mnist_0361.png', 'mnist_0075.png', 'mnist_0773.png', 'mnist_0821.png', 'mnist_0128.png', 'mnist_0537.png', 'mnist_0816.png', 'mnist_0068.png', 'mnist_0411.png', 'mnist_0865.png', 'mnist_0065.png', 'mnist_0523.png', 'mnist_0406.png', 'mnist_0794.png', 'mnist_05

In [22]:
!cp generated_images/*.png generated_images/fake/

In [24]:
class InceptionV3FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        weights = Inception_V3_Weights.IMAGENET1K_V1
        self.model = inception_v3(weights=weights)  # ✅ FIX
        self.model.eval()

        for p in self.model.parameters():
            p.requires_grad = False

    def forward(self, x):
        with torch.no_grad():
            if x.shape[-1] != 299:
                x = F.interpolate(
                    x, size=(299, 299),
                    mode="bilinear", align_corners=False
                )

            x = self.model.Conv2d_1a_3x3(x)
            x = self.model.Conv2d_2a_3x3(x)
            x = self.model.Conv2d_2b_3x3(x)
            x = self.model.maxpool1(x)
            x = self.model.Conv2d_3b_1x1(x)
            x = self.model.Conv2d_4a_3x3(x)
            x = self.model.maxpool2(x)
            x = self.model.Mixed_5b(x)
            x = self.model.Mixed_5c(x)
            x = self.model.Mixed_5d(x)
            x = self.model.Mixed_6a(x)
            x = self.model.Mixed_6b(x)
            x = self.model.Mixed_6c(x)
            x = self.model.Mixed_6d(x)
            x = self.model.Mixed_6e(x)
            x = self.model.Mixed_7a(x)
            x = self.model.Mixed_7b(x)
            x = self.model.Mixed_7c(x)
            x = self.model.avgpool(x)

            return x.view(x.size(0), -1)  # [B, 2048]



# ---------------------------------------------
# Image Loader (expects subfolders)
# ---------------------------------------------
def get_image_loader(folder_path, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])

    dataset = datasets.ImageFolder(folder_path, transform=transform)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)


# ---------------------------------------------
# Extract activations
# ---------------------------------------------
def get_activations(loader, model, device):
    feats = []
    for imgs, _ in tqdm(loader, desc="Extracting features"):
        imgs = imgs.to(device)
        feats.append(model(imgs).cpu().numpy())
    return np.concatenate(feats, axis=0)


# ---------------------------------------------
# Frechet Distance
# ---------------------------------------------
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2):
    diff = mu1 - mu2
    covmean, _ = linalg.sqrtm(sigma1 @ sigma2, disp=False)

    if np.iscomplexobj(covmean):
        covmean = covmean.real

    return diff @ diff + np.trace(sigma1 + sigma2 - 2 * covmean)


# ---------------------------------------------
# Main FID function
# ---------------------------------------------
def compute_fid_from_folders(real_folder, gen_folder, device):
    real_loader = get_image_loader(real_folder)
    gen_loader = get_image_loader(gen_folder)

    model = InceptionV3FeatureExtractor().to(device)

    real_feats = get_activations(real_loader, model, device)
    gen_feats = get_activations(gen_loader, model, device)

    mu1, sigma1 = real_feats.mean(0), np.cov(real_feats, rowvar=False)
    mu2, sigma2 = gen_feats.mean(0), np.cov(gen_feats, rowvar=False)

    return calculate_frechet_distance(mu1, sigma1, mu2, sigma2)


# ---------------------------------------------
# Run
# ---------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    fid = compute_fid_from_folders(
        real_folder="real_images",
        gen_folder="generated_images",
        device=device
    )

    print(f"FID Score: {fid:.4f}")


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


100%|██████████| 104M/104M [00:00<00:00, 123MB/s] 
Extracting features: 100%|██████████| 34/34 [00:05<00:00,  6.16it/s]
/tmp/ipython-input-1380039855.py:73: DeprecationWarning: The `disp` argument is deprecated and will be removed in SciPy 1.18.0.
  covmean, _ = linalg.sqrtm(sigma1 @ sigma2, disp=False)


FID Score: 48.3179


In [25]:
current_dir = os.getcwd()
folders = [f for f in os.listdir(current_dir) if os.path.isdir(os.path.join(current_dir, f))]

print("📁 Folders in current directory:")
for folder in folders:
    print(f" - {folder}")

📁 Folders in current directory:
 - .config
 - generated_images
 - data
 - MNIST
 - real_images
 - sample_data


In [26]:
import os
import shutil

def wrap_in_class_folder(base_dir, class_name="class_x"):
    class_dir = os.path.join(base_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

    for fname in os.listdir(base_dir):
        path = os.path.join(base_dir, fname)
        if os.path.isfile(path):
            shutil.move(path, os.path.join(class_dir, fname))

# Example usage
wrap_in_class_folder("real_images")
wrap_in_class_folder("generated_images")

In [27]:
import shutil
import os

for folder in ["real_images", "generated_images"]:
    checkpoints = os.path.join(folder, ".ipynb_checkpoints")
    if os.path.exists(checkpoints):
        shutil.rmtree(checkpoints)
        print(f"Removed: {checkpoints}")